# **Projeto Algoritmos Genéticos**
## Problema N-rainhas
### - Posicionar as *N* rainhas no tabuleiro sem que uma elimine a outra, ou seja, as peças não podem estar na mesma linha, coluna ou diagonal.
### - Especificações:


*   Quantidade de rainhas
*   Taxas: Mutação e Cruzamento
*   Tamanhos: população, eletismo e seleção
*   Critério de parada: Número de iterações e ou solução ótima



In [ ]:
#Gustavo Luiz Conceição Zago RA: 2268221 // Giovanni Henrique Munhoz De Lion Siervo RA: 2144255

In [ ]:
from numpy.random import *


In [ ]:
#População gerada aleatoriamente
# size: tamanho do cromossomo
def random_chromosome(size):
    return [ randint(1, size) for _ in range(size) ]

In [ ]:
# Geração de valores aleatórios para teste
# individual: Um cromossomo qualquer da população
# Função objetivo: número de pares de rainhas não-atacantes
# Exemplo: 8 rainhas
# min = 0, max = 8 x 7/2 = 28

def fitness(individual):
    # max de rainhas não atacáveis
    h = (len(individual)*7)/2
    #Verificação de ataques
    for i in range(0, len(individual)):
        for j in range(0, len(individual)):
            if j > i:
               if abs(i - j) == abs(individual[i] - individual[j]): #valor abs de linha e coluna deve ser igual à posição da rainha
                  h -= 1
                  #ataques em coluna
               if abs(individual[i] - individual[j]) == 0:
                  h -= 1
    return h

In [ ]:
# Seleção por torneio
# population: População atual
# sizek: tamanho do torneio

def selection(population, sizeK):
  # seleciona um indivíduo aleatório
	selection_ix = randint(len(population))
  # seleciona um conjunto aleatório de indivíduos
	for ix in randint(0, len(population), sizeK-1):
		# compara com outros (realiza o torneio), se um dos indivíduos selecionados do conjunto for melhor que o pré selecionado
		# seu índice é armazenado em selection_ix
		if population[ix][1] > population[selection_ix][1]:
			selection_ix = ix
	# retorna o melhor indivíduo como pai para a proxima geração
	return population[selection_ix][0]

In [ ]:
#Cruzamento de 1 ponto aleatório
# p1 e p2: dois individuos da população
# r_cross: Taxa de cruzamento

def crossover(p1, p2, r_cross):
	# cópia dos pais
	c1, c2 = p1.copy(), p2.copy()
	# checagem de cruzamento
	if random() < r_cross:
		# seleção do ponto aleatório para o cruzamento (seleção da quebra)
		pt = randint(1, len(p1)-1)
		# cruzamento (concatena as partes separadas das soluções)
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

In [ ]:
# Mutação bit a bit
# individual: Um cromossomo qualquer da população
# r_mut: Taxa de mutação
def mutation(individual, r_mut, nqueens):
  for i in range(len(individual)):
    # checagem de mutação
    if random() < r_mut:
      inicial = individual[i]
      # garantia da mutação
      while inicial == individual[i]:
        # muta a posição para outra posição aleatória
        individual[i] = randint(1, nqueens+1)

In [ ]:
# Elitismo
#pop: População atual
#best_ind: Melhores individuos selecionados
#          (tamanho definido por parâmetro)
# ... Se for necessário mais parâmetros
#def elitism(pop, best_ind,..):

def elitism(pop, best_ind):
  # chama função fitness para avaliação dos indivíduos
  scores = [fitness(p) for p in pop]
	# cria população temporária com fit atualizado
  newPop = []
  for i in zip(pop,scores):
      newPop.append(list(i))

  # concatena população com o melhor indivíduo salvo anteriormente
  newPop = newPop + best_ind
  # ordena de melhor para pior
  newPop.sort(key = lambda newPop: newPop[:][1], reverse = True)
  # remove os piores indivíduos da população
  newPop = newPop[:-(len(best_ind))]
  # limpa a lista da população original
  pop.clear()
  # adiciona a população atualizada na lista da população original
  for i in newPop:
    pop.append(i[0])
  return pop

In [ ]:
# Salva os melhores indivíduos com base em elt
def saveBestInd(pop,sizeInd):
  # cria lista para armazenar os melhores individuos
  copyBest = []
  # ordena de melhor para pior com base no fitness (pop[:][1])
  pop.sort(key = lambda pop: pop[:][1], reverse=True)
  # copia os elt primeiros indivíduos para a lista
  copyBest = pop[:sizeInd].copy()
  return copyBest

In [ ]:
# Recolhe a melhor solução
def bestSolution(pop):
  # ordena de melhor para pior com base no fitness (pop[:][1])
    pop.sort(key = lambda pop: pop[:][1], reverse=True)
    # retorna o primeiro indivíduo
    return pop[0][0],pop[0][1]

In [ ]:
# Poderá ser utlizado qualquer quantidade de parâmetros
def genetic_algorithm(n_queens, n_iter, n_pop, r_cross, r_mut, k, sizeBestInd, optimalSol):
	# Pop: data structure used:
	##### index [0] -> numero de posições (1-8) (indivíduo)
	##### index [1] -> fitness
	pop = []
	# lista para elitismo
	bestInd = []
  # população inicial de indivíduos com posições aleatórias
	tempPop = [randint(1, n_queens+1, n_queens).tolist() for _ in range(n_pop)]
	# enumeração das gerações
	for gen in range(n_iter):
			# avalia o fit dos indivíduos
			fit = [fitness(chrom) for chrom in tempPop]
			# concatena com pop
			for i in zip(tempPop, fit):
				pop.append(list(i))
		  # acompanhamento da melhor solução da geração
			ind, best_eval = bestSolution(pop)
			print("Geração: ",gen," Ind: ",ind," Fit: ",best_eval)
			# checa se a melhor solução possível foi encontrada
			if best_eval == optimalSol:
				print(">>>>>> Melhor solução geração: %d" % gen)
				print(">>>>>> ",ind," - ",best_eval)
				break
			# aplica elitismo (salva os melhores elt indivíduos)
			bestInd = saveBestInd(pop, sizeBestInd)
			# executa o torneio entre os k
			selected = [selection(pop, k) for _ in range(tam_pop)]
			# cria a próxima geração (filhos)
			children = []
			# limpa a população atual
			pop.clear()
			for i in range(0, n_pop, 2):
				# seleciona os pais de 2 em 2
				p1, p2 = selected[i], selected[i+1]
				# executa o cruzamento
				for c in crossover(p1, p2, r_cross):
					# executa mutação
					mutation(c, r_mut, n_queens)
					# salva para a próxima geração
					children.append(c)
			# subistui a população pelos filhos
			tempPop = children
			# concatena a população atual com elitismo
			tempPop = elitism(tempPop, bestInd)
	return tempPop

In [ ]:
def print_population(pop):
    for i in pop:
      print("Cromossomo: ",i[0]," - Fit: ",i[1])

In [ ]:
pop = []

#tamanho da população
tam_pop = 50
# numero de rainhas
nqueens = 8
# numero de iterações
n_iter = 70
# taxa de sucesso cruzamento
r_cross = 0.9
# taxa de sucesso mutação
r_mut = 0.2
# numero de indivíduos no torneio
k = 16
# melhor solução possível
x = nqueens * 7/2
# numero de indivíduos no elitismo
elt = 5

# executa busca por algoritmo genético
pop = genetic_algorithm(nqueens, n_iter, tam_pop, r_cross, r_mut, k, elt,x)
# avalia todos os candidatos da população
scores = [fitness(p) for p in pop]
# concatena pop com scores
print("População Final:")
pop_with_fitness = [(pop[i], scores[i]) for i in range(len(pop))]
print_population(pop_with_fitness)

#print("População Final:")
#for i in zip(pop,scores):
#    print(i)

Geração:  0  Ind:  [1, 8, 6, 2, 4, 1, 8, 5]  Fit:  25.0
Geração:  1  Ind:  [4, 7, 5, 3, 6, 6, 4, 2]  Fit:  25.0
Geração:  2  Ind:  [1, 8, 6, 2, 4, 2, 8, 5]  Fit:  25.0
Geração:  3  Ind:  [1, 8, 6, 3, 7, 2, 8, 1]  Fit:  25.0
Geração:  4  Ind:  [1, 8, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  5  Ind:  [1, 8, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  6  Ind:  [1, 1, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  7  Ind:  [3, 1, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  8  Ind:  [1, 1, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  9  Ind:  [1, 3, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  10  Ind:  [1, 1, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  11  Ind:  [1, 1, 6, 2, 7, 2, 8, 5]  Fit:  26.0
Geração:  12  Ind:  [1, 8, 6, 2, 7, 2, 8, 3]  Fit:  26.0
Geração:  13  Ind:  [1, 1, 6, 2, 7, 2, 8, 3]  Fit:  26.0
Geração:  14  Ind:  [1, 1, 6, 2, 7, 2, 8, 3]  Fit:  26.0
Geração:  15  Ind:  [1, 1, 6, 2, 7, 2, 8, 3]  Fit:  26.0
Geração:  16  Ind:  [3, 1, 6, 2, 5, 2, 8, 3]  Fit:  26.0
Geração:  17  Ind:  [1, 1, 6, 2, 7, 2, 8,